# Asignement 3 
By Māra Učelniece 

### Task : 
Evaluate how varying structures relate to policy and in which cases they might apply. 

- Response : there is not a specific data set we have to use, but just apply the evaluation of the structures to the EV's case that we are attempting to create policy interventions for. 
 
--------------
1. Evaluate the current code 

    1.1. Go triugh the read me 
    
    1.2. Run the experiment 
    
    1.3. Evaluate the plotting functions 
    
2. Start on model evalautaion 

    2.1. See what can be reused 
    
    2.2. Start with one model gen/ evaluation 
    
        2.2.1. Report on that 
        
    2.3. Do with the other models 
    
3. Start on policy interventions 

    3.1. Subsadies 
    
    3.2. Targeted seeding 
----------------

#### Questions : 
- Do we want to find a data set ? 

In [1]:
# Imports 
    # Copied from exmaple notebook 
    
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
import networkx as nx
import numpy as np
import random
from typing import Iterable, List, Dict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed

C:\Users\Mara\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Mara\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [ ]:
# Generate the models 


In [ ]:
# Evaluate trough plots 
